# <center>JSON Output Parser

In [57]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [48]:
load_dotenv()

True

In [49]:
repo_id="google/gemma-2-2b-it"

In [50]:
llm = HuggingFaceEndpoint(
    repo_id = repo_id,
    task="text-generation",
    verbose=True,
    huggingfacehub_api_token=os.environ['HUGGINGFACE_ACCESS_TOKEN']
)

In [51]:
model = ChatHuggingFace(llm=llm)

In [52]:
parser = JsonOutputParser()

In [53]:
template = PromptTemplate(
    template="Give me the name, age and city of frictional person from the game {game_name} \n {format_instruction}",
    input_variables=['game_name'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)


In [93]:
# prompt = template.format(game_name="Red Dead Redemption 2")
prompt = template.format(game_name="GTA V")

In [94]:
prompt

'Give me the name, age and city of frictional person from the game GTA V \n Return a JSON object.'

In [95]:
# -x-x-x-x-x-x

from langchain_google_genai import ChatGoogleGenerativeAI



API_KEY = os.environ['GEMINI_API_KEY']
MODEL_NAME = "gemini-2.0-flash"
model = ChatGoogleGenerativeAI(api_key=API_KEY, model=MODEL_NAME)


# -x-x-x-x-x-x

In [96]:
result = model.invoke(prompt)

In [97]:
result.content

'```json\n{\n  "name": "Michael De Santa",\n  "age": "Mid-40s",\n  "city": "Los Santos"\n}\n```'

In [98]:
result = parser.parse(result.content)

In [99]:
type(result)

dict

In [100]:
result

{'name': 'Michael De Santa', 'age': 'Mid-40s', 'city': 'Los Santos'}

# <center> Same Code Using Chain

In [102]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_core.output_parsers import JsonOutputParser

In [103]:
load_dotenv()

True

In [107]:
API_KEY = os.environ['GEMINI_API_KEY']
MODEL_ID = "gemini-2.0-flash"

In [108]:
model = ChatGoogleGenerativeAI(api_key=API_KEY, model=MODEL_ID, temperature=1)

In [109]:
model

ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=1.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000289563A5A20>, default_metadata=(), model_kwargs={})

In [111]:
parser = JsonOutputParser()

In [119]:
template = PromptTemplate(
    template="Give me the name, age and city of frictional person from the game {game_name} \n {format_instruction}",
    input_variables=['game_name'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

In [120]:
chain = template | model | parser

In [121]:
chain

PromptTemplate(input_variables=['game_name'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='Give me the name, age and city of frictional person from the game {game_name} \n {format_instruction}')
| ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', google_api_key=SecretStr('**********'), temperature=1.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000289563A5A20>, default_metadata=(), model_kwargs={})
| JsonOutputParser()

In [125]:
chain.invoke({"game_name": "GTA Vice City"})

{'name': 'Tommy Vercetti', 'age': 35, 'city': 'Vice City'}